In [1]:
# Librerias ----
library(tidyverse) 
library(wooldridge)
library(stargazer) # Resultados regresiones
library(AER) # Resultados bootstrap

Warning message:
"package 'tidyverse' was built under R version 4.0.5"
-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.7
v tidyr   1.1.4     v stringr 1.4.0
v readr   2.1.0     v forcats 0.5.1

Warning message:
"package 'ggplot2' was built under R version 4.0.5"
Warning message:
"package 'tidyr' was built under R version 4.0.5"
Warning message:
"package 'purrr' was built under R version 4.0.5"
Warning message:
"package 'dplyr' was built under R version 4.0.5"
Warning message:
"package 'stringr' was built under R version 4.0.5"
Warning message:
"package 'forcats' was built under R version 4.0.5"
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Warning message:
"package 'stargazer' was built under R

# Optimizadores

## Newton Raphson

In [2]:
# Funcion para llevar a cabo estimacion probit por Newton Raphson
# metodo: función con los objetos del modelo en cuestión
# Y: variable dependiente binaria. Matriz de Nx1
# X: matriz NxK de variables explicativas
# b0: matriz de Kx1 con valores iniciales para los parametros del modelo
# max_iter: cantidad maxima de iteraciones a realizar. Si no se converge en esta cantidad de iteraciones el algoritmo se detiene
# tol: diferencia entre las actualizaciones de los parametros que se tolera para considerar convergencia. Debe ser un numero pequeño
# print_iter: si es TRUE imprime en la consola informacion cada 500 iteraciones. Si es FALSE queda en silencio
newton_raphson <- function(metodo, Y, X, b0, tol = 1e-8, max_iter=500, print_iter = TRUE){
   
   # Valores de beta iniciales y objeto auxiliar para guardar actualizaciones
   beta <- b0
   beta_new <- 0
   
   # Valor para diferencia. Cualquier valor mayor a 'tol' para comenzar
   beta_diff <- 100
   
   # Contador de iteraciones
   i <- 0
   
   # NR
   while (beta_diff >= tol & i <= max_iter) {
      
      # Objetos del modelo
      objetos <- metodo(Y,X,beta)
      J <- objetos$J
      H <- objetos$H
      loglik <- objetos$loglik
      
      # Actualizamos
      beta_new  <- beta -  (pracma::inv(H) %*% J)
      beta_diff <- max(beta_new - beta)
      beta <- beta_new
      
      # Esto imprime resultados intermedios en la consola
      if (print_iter){
         cat(paste0("Iter: ", i, " , logLik: ", loglik, " , Beta Diff: ", beta_diff, "\n"))
      }
      
      # Actualiza el contador de iteraciones
      i <- i + 1
      
   }
   
   # Da aviso de que el algoritmo se ha detenido porque se alcanzo la cantidad maxima de iteraciones
   if (i == max_iter){
      cat("-- Maximum of iterations reached! --\n")
   }
   
   output <- list(beta = as.numeric(beta), loglik = loglik, varcov = pracma::inv(-H), xb = objetos$xb)
   return(output)
}

## Gradient Descent

In [3]:
# Funcion para llevar a cabo estimacion probit por Gradient  Descent
# metodo: función con los objetos del modelo en cuestión
# Y: variable dependiente binaria. Matriz de Nx1
# X: matriz NxK de variables explicativas
# b0: matriz de Kx1 con valores iniciales para los parametros del modelo
# max_iter: cantidad maxima de iteraciones a realizar. Si no se converge en esta cantidad de iteraciones el algoritmo se detiene
# tol: diferencia entre las actualizaciones de los parametros que se tolera para considerar convergencia. Debe ser un numero pequeño
# lr: learning rate, valor pequeño por el cual se 'castiga' el gradiente
# print_iter: si es TRUE imprime en la consola informacion cada 250 iteraciones. Si es FALSE queda en silencio
gradient_descent <- function(metodo, Y, X, b0, tol = 1e-9, lr = 0.001, max_iter=2500, print_iter = TRUE){
   
   # Valores de beta iniciales y objeto auxiliar para guardar actualizaciones
   beta <- b0
   beta_new <- 0
   
   # Valor para diferencia. Cualquier valor mayor a 'tol' para comenzar
   beta_diff <- 100
   
   # Contador de iteraciones
   i <- 1
    
   # Para X con constante en ultima columna, escala las variables
   ncols <- ncol(X)
   X_scaled <- cbind(scale(X[,1:(ncols-1)]), cons=1)
   
   # GD
   while (beta_diff >= tol & i <= max_iter) {
      
      ## Objetos del modelo
      objetos <- metodo(Y,X_scaled,beta)
      J <- objetos$J
      loglik <- objetos$loglik
      
      # Actualizamos
      beta_new  <- beta +  (lr * J)
      beta_diff <- max(beta_new - beta)
      beta <- beta_new
      
      # Esto imprime resultados intermedios en la consola
      if (print_iter & i %% 250 == 0){
         cat(paste0("Iter: ", i, " , logLik: ", loglik, " , Beta Diff: ", beta_diff, "\n"))
      }
      
      # Actualiza el contador de iteraciones
      i <- i + 1
      
   }
   
   # Da aviso de que el algoritmo se ha detenido porque se alcanzo la cantidad maxima de iteraciones
   if (i == max_iter){
      cat("-- Maximum of iterations reached! --\n")
   }
   
   # Quita el scale a los betas
   desv_estandar_xs <- apply(X[,1:(ncols-1)], 2, sd)
   promedio_xs      <- apply(X[,1:(ncols-1)], 2, mean)
   
   nombres   <- rownames(beta)
   betas_x   <- beta[1:(ncols-1)] / desv_estandar_xs %>% as.matrix()
   beta_cons <- beta[ncols] - sum(promedio_xs * betas_x) %>% as.matrix()
   beta <- rbind(betas_x, beta_cons)
   rownames(beta) <- nombres 
    
   output <- list(beta = beta, loglik = loglik, xb = objetos$xb)
   return(output)
   
}

## Bootstrap: SE de coeficientes
Si no hemos podido obtener la matriz Hessiana, y así la matriz de VarCov, podemos obtener errores estándar mediante Bootstrap (podría combinarse con el uso de Gradient Descent)

In [4]:
# Función para obtener coeficientes estimados y errores estándar mediante Bootstrap
# metodo: función con los objetos del modelo en cuestión 
# Y: variable dependiente binaria. Matriz de Nx1
# X: matriz NxK de variables explicativas
# b0: matriz de Kx1 con valores iniciales para los parametros del modelo
# max_iter: cantidad maxima de iteraciones a realizar. Si no se converge en esta cantidad de iteraciones el algoritmo se detiene
# tol: diferencia entre las actualizaciones de los parametros que se tolera para considerar convergencia. Debe ser un numero pequeño
# lr: learning rate, valor pequeño por el cual se 'castiga' el gradiente
# print_iter: si es TRUE imprime en la consola informacion cada 250 iteraciones. Si es FALSE queda en silencio
bootstrap_coefs_se <- function(metodo, Y, X, b0, seed=04051996, reps = 10, parallel = "snow", 
                               max_iter = 2500, lr = 0.001, tol = 1e-9, print_iter = FALSE){
   
   # Función para iterar en 'boot'
   boot_func <- function(datos, indices){
      
      data <- datos[indices, ]
      ncols <- ncol(data)
      y <- data[1] %>% as.matrix()
      x <- data[2:ncols] %>% as.matrix()
      
      resultado <- gradient_descent(metodo, y, x, b0, max_iter = max_iter, lr = lr, tol = tol, print_iter = print_iter)
      return(resultado$beta)
      
   }
   
   # Ajustamos datos para función
   datos <- cbind(Y,X) %>% as.data.frame()
   
   # Bootstrap
   set.seed(seed)
   result_boot <- boot::boot(data = datos, statistic = boot_func, R = reps, parallel = parallel)
   
   # En tabla
   variables <- colnames(result_boot$data)[-1]
   tabla <- summary(result_boot) %>% as_tibble() %>% 
      mutate(Variable = variables) %>% select(Variable, original, bootSE) %>% as.data.frame()
   
   return(tabla)
      
}

# Probit

\begin{align*}
    \text{(Función de log-verosimilitud)  } \log \mathcal{L}_N(\beta ; y|x) &= \sum_{i=1}^{N} y_i \log\Phi(\beta_0 + \beta_1 x_i) + (1-y_i) \log(1-\Phi(\beta_0 + \beta_1 x_i)) \\
    &= \text{sum}\left( \mathbf{Y \circ \log\Phi + (1-Y) \circ \log[1-\Phi] } \right) \\
    \text{(Gradiente)  } \mathbf{J(\hat{\beta})} &= \mathbf{X' [Y \circ (\hat{\phi} \oslash \hat{\Phi})]} - \mathbf{X' [ (1 - Y) \circ (\hat{\phi} \oslash (1-\hat{\Phi}))]} \\
    \text{(Matriz Hessiana)  } \mathbf{\hat{H}} &= \mathbf{X' \text{Diag}(- \hat{\phi} \circ [Y \circ \hat{A} + (1 - Y) \circ \hat{B}]) X}
\end{align*}

donde 
\begin{align*}
    \mathbf{A} &= [\phi + \mathbf{X \beta} \circ \Phi] \oslash [\Phi \circ \Phi] \\
            \mathbf{B} &= [\phi - \mathbf{X \beta} \circ (1-\Phi)] \oslash [(1-\Phi) \circ (1-\Phi)]
\end{align*}

Con matrices (para el caso de regresión simple $k=2$, generalizable a todo $k$)
\begin{align*}
            \mathbf{X} = \begin{bmatrix}
                1 & x_1 \\
                \vdots & \vdots\\
                1 & x_N
            \end{bmatrix} , 
            \mathbf{Y} = \begin{bmatrix}
                y_1 \\ \vdots \\ y_N
            \end{bmatrix} , 
            \phi = \begin{bmatrix}
                \phi(\beta_0 + \beta_1 x_1) \\ \vdots \\ \phi(\beta_0 + \beta_1 x_N) \end{bmatrix} ,
            \Phi = \begin{bmatrix}
                \Phi(\beta_0 + \beta_1 x_1) \\ \vdots \\ \Phi(\beta_0 + \beta_1 x_N) 
            \end{bmatrix}
        \end{align*}
**Escribamos estas ecuaciones en R para resolver el problema de optimización**

In [5]:
# Método para modelo Probit
probitFuncs <- function(Y, X, beta){
   
   # Matriz Nx1 de valores predichos, dado beta
   xb  <- X %*% beta # Esto sería en el ejemplo simple b0 + b1*x1
   Phi <- pnorm(xb) # pnorm es la función de probabilidad acumulada normal estándar
   phi <- dnorm(xb) # dnorm es la función de densidad normal estándar
   
   # Funcion de verosimilitud
   loglik <- sum(Y * log(Phi) + (1-Y) * log(1-Phi))
   
   # Gradiente, dado beta
   gradiente <- t(X) %*% (Y * (phi / Phi)) - t(X) %*% ((1 - Y) * (phi / (1-Phi)))
   
   # Hessiano, dado beta
   A    <- (phi + xb * Phi) / Phi^2
   B    <- (phi - xb * (1-Phi)) / (1-Phi)^2
   S    <- as.numeric(- phi * ( Y*A + (1-Y)*B ))
   hess <- t(X) %*% diag(S) %*% X
   
   output <- list(loglik = loglik, J = gradiente, H = hess, xb = xb)
   return(output)
}   

# Probando nuestros algoritmos

## Usando las librerías de R
Para comparar, usemos la función que trae R para resolver Probit


In [6]:
# Importamos la base de datos
data <- mroz

modelo_probit <- glm(inlf ~ nwifeinc + educ + exper + I(exper^2) + age + kidslt6 + kidsge6,
                     data = data, family = binomial(link = "probit"),
                     control = list(epsilon = 1e-12))

summary(modelo_probit)


Call:
glm(formula = inlf ~ nwifeinc + educ + exper + I(exper^2) + age + 
    kidslt6 + kidsge6, family = binomial(link = "probit"), data = data, 
    control = list(epsilon = 1e-12))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2156  -0.9151   0.4315   0.8653   2.4553  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.2700768  0.5080923   0.532  0.59504    
nwifeinc    -0.0120237  0.0049392  -2.434  0.01492 *  
educ         0.1309047  0.0253995   5.154 2.55e-07 ***
exper        0.1233476  0.0187590   6.575 4.85e-11 ***
I(exper^2)  -0.0018871  0.0005999  -3.145  0.00166 ** 
age         -0.0528527  0.0084627  -6.245 4.23e-10 ***
kidslt6     -0.8683285  0.1183820  -7.335 2.22e-13 ***
kidsge6      0.0360050  0.0440316   0.818  0.41352    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1029.7  on 752  degrees of freedom
Residual dev

## Antes de probar nuestros algoritmos
En las funciones que hemos creado insertamos la variable dependiente ($\mathbf{Y}$) y las variables explicativas ($\mathbf{X}$) en matrices ($N\times 1$ y $N\times K$, respectivamente).

Adicionalmente, requerimos un valor inicial $\beta_0$ para inicial los métodos de solución. 

In [7]:
# Nombre de la variable dependiente entre comillas
variable_dependiente <- "inlf"

# Nombres de las variables independientes entre comillas en un vector
variables_independientes <- c("nwifeinc", "educ", "exper", "expersq", "age", "kidslt6", "kidsge6")

# Matriz Nx1 con variable dependiente
Y <- data %>% select(all_of(variable_dependiente)) %>% as.matrix()
head(Y)

# Matriz NxK con variables explicativas. Se estandariza y se le agrega una constante
X <- data %>% select(all_of(variables_independientes)) %>% 
   as.matrix() %>% cbind("cons"=1)
head(X)

# Valor inicial. Todos los beta (beta0 hasta b7 con valor 0.0005)
b0 <- rep(.0005, 8)

,inlf
1,1
2,1
3,1
4,1
5,1
6,1


,nwifeinc,educ,exper,expersq,age,kidslt6,kidsge6,cons
1,10.910060,12,14,196,32,1,0,1
2,19.499981,12,5,25,30,0,2,1
3,12.039910,12,15,225,35,1,3,1
4,6.799996,12,6,36,34,0,3,1
5,20.100058,14,7,49,31,1,2,1
6,9.859054,12,33,1089,54,0,0,1


## Probando Newton Raphson

In [8]:
# Estimacion probit Newton Raphson
resultado_nr <- newton_raphson(probitFuncs, Y, X, b0)

# Errores estandar de los coeficientes asociados a las variables explicativas
varianzas <- diag(resultado_nr$varcov) # Obtenemos matriz de varianzas y covarianzas
errores_estandar <- sqrt(varianzas) # Obtiene errores estandar como la raíz cuadrada de cada varianza

# Coeficientes y errores estandar en una tabla
data.frame(beta = resultado_nr$beta, error_estandar = errores_estandar)

# Comparación libreria R
message("Comparación")
summary(modelo_probit)

Iter: 0 , logLik: -499.349477100748 , Beta Diff: 0.217443176590716
Iter: 1 , logLik: -405.243048268953 , Beta Diff: 0.0477988576696768
Iter: 2 , logLik: -401.323901552511 , Beta Diff: 0.0043010497303283
Iter: 3 , logLik: -401.302193977913 , Beta Diff: 3.36870958276281e-05
Iter: 4 , logLik: -401.302193173895 , Beta Diff: 1.54869178592421e-09


,beta,error_estandar
,<dbl>,<dbl>
nwifeinc,-0.01202374,0.0048398383
educ,0.13090473,0.0252541957
exper,0.12334759,0.0187164015
expersq,-0.00188708,0.0005999864
age,-0.05285267,0.0084772396
kidslt6,-0.86832851,0.1185223109
kidsge6,0.03600496,0.0434767876
cons,0.27007677,0.5085930353


Comparación




Call:
glm(formula = inlf ~ nwifeinc + educ + exper + I(exper^2) + age + 
    kidslt6 + kidsge6, family = binomial(link = "probit"), data = data, 
    control = list(epsilon = 1e-12))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2156  -0.9151   0.4315   0.8653   2.4553  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.2700768  0.5080923   0.532  0.59504    
nwifeinc    -0.0120237  0.0049392  -2.434  0.01492 *  
educ         0.1309047  0.0253995   5.154 2.55e-07 ***
exper        0.1233476  0.0187590   6.575 4.85e-11 ***
I(exper^2)  -0.0018871  0.0005999  -3.145  0.00166 ** 
age         -0.0528527  0.0084627  -6.245 4.23e-10 ***
kidslt6     -0.8683285  0.1183820  -7.335 2.22e-13 ***
kidsge6      0.0360050  0.0440316   0.818  0.41352    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1029.7  on 752  degrees of freedom
Residual dev

## Probando Gradient Descent

In [9]:
# Estimacion probit Gradient Descent
resultado_gd   <- gradient_descent(probitFuncs, Y, X, b0)

# Estimacion errores estandar por bootstrap. Note que para todo esto no necesitamos la matriz Hessiana
message("Calculando errores estándar ...")
boot_result_se <- bootstrap_coefs_se(probitFuncs, Y, X, b0)
message("Terminado")

Iter: 250 , logLik: -401.302294079722 , Beta Diff: 4.8437573343163e-05
Iter: 500 , logLik: -401.302193174762 , Beta Diff: 1.42004542458274e-07


Calculando errores estándar ...

Terminado



In [10]:
boot_result_se

Variable,original,bootSE
<chr>,<dbl>,<dbl>
nwifeinc,-0.01202374,0.0047582056
educ,0.13090473,0.0191909735
exper,0.12334759,0.0210234561
expersq,-0.00188708,0.0006035634
age,-0.05285267,0.0078138858
kidslt6,-0.86832851,0.1117828927
kidsge6,0.03600496,0.0231652487
cons,0.27007680,0.4628145322
